In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Exploration

## 1.1 Imports

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import math

## 1.2 Read CSV

In [ ]:
df = pd.read_csv("/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv")

## 1.3 Basic data structure exploration

In [ ]:
print(f'Shape of heart.csv: {df.shape}')

There are 303 records in the dataset, and 14 features to describe them.

In [ ]:
print(f'\nFirst 5 rows of heart.csv:')
df.head()

In [ ]:
print(f'Feature information:')
df.info()

In [ ]:
print('Number of null values in each column:')
df.isnull().sum()

As there are no null values in the data, null-filling will not be necessary in the data preparation.

In [ ]:
print('Number of unique values in each column:')
df.nunique(axis=0)

In [ ]:
print('Variance in each column:')
df.var(axis=0)

With such a wide range of variance across the columns, some features, particularly the continuous features, will require scaling.

## 1.4 Categorical feature exploration

### List of categorical features

In [ ]:
CATEGORICAL_FEATURES = ['sex', 'cp', 'fbs', 'restecg', 'exng', 'slp', 'thall', 'caa']

In [ ]:
df_ = pd.DataFrame()
for column in CATEGORICAL_FEATURES:
    df_[column] = df[column].astype("category")

print(f'Basic categorical feature statistical information:')
df_[CATEGORICAL_FEATURES].describe()

In [ ]:
rows = 4
columns = math.ceil(len(CATEGORICAL_FEATURES)/rows)
fig, ax = plt.subplots(
    rows,
    columns,
    figsize=(3*columns, 3*rows),
    sharey=False,
)
for i, column in enumerate(CATEGORICAL_FEATURES):
    plot = sns.countplot(
        data=df, 
        x=column, 
        ax=ax[i%rows, i//rows], 
        color=sns.color_palette('deep')[0],
    )
    plot.set_xlabel(column, fontsize=14)
    if i//rows > 0:
        plot.set(ylabel=None)
fig.suptitle('Count Plot for categorical features', fontsize=16)
fig.tight_layout()

Notes:
* There are nearly twice as many of sex category 1 (male) as there are of category 0 (female).

In [ ]:
rows = 4
columns = math.ceil(len(CATEGORICAL_FEATURES)/rows)
fig, ax = plt.subplots(
    rows,
    columns,
    figsize=(4*columns, 4*rows),
    sharey=False,
)
for i, column in enumerate(CATEGORICAL_FEATURES):
    plot = sns.countplot(
        data=df, 
        x=column, 
        hue='output', 
        ax=ax[i%rows, i//rows], 
        palette=sns.color_palette('deep'),
    )
    plot.set_xlabel(column, fontsize=14)
    if i//rows > 0:
        plot.set(ylabel=None)
fig.suptitle('Count plot for categorical features split by target', fontsize=16)
fig.tight_layout()

Notes:
* Samples of sex category 0 (female) are much more likely to have a target output of 1 (heart attack), especially as compared to sex category 1 (male).
* exng 0, cp 1 & 2, slp 2, thall 2, and caa 0 have high rates of heart attack.

## 1.5 Continuous feature exploration

### List of continuous features

In [ ]:
CONTINUOUS_FEATURES = ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']

In [ ]:
print(f'Basic continuous feature statistical information:')
df[CONTINUOUS_FEATURES].describe()

In [ ]:
rows = 3
columns = math.ceil(len(CONTINUOUS_FEATURES)/rows)
fig, ax = plt.subplots(
    rows,
    columns,
    figsize=(4*columns, 4*rows),
    sharey=False,
)
ax[-1, -1].axis('off')
for i, column in enumerate(CONTINUOUS_FEATURES):
    plot = sns.histplot(
        data=df, 
        x=column, 
        ax=ax[i%rows, i//rows], 
        linewidth=0,
        color=sns.color_palette('deep')[0],
    )
    plot.set_xlabel(column, fontsize=14)
    if i//rows > 0:
        plot.set(ylabel=None)
fig.suptitle('Histogram for each continuous feature', fontsize=16)
fig.tight_layout()

In [ ]:
rows = 2
columns = math.ceil(len(CONTINUOUS_FEATURES)/rows)
fig, ax = plt.subplots(
    rows, 
    columns,
    figsize=(2*columns, 5*rows),
    sharey=False,
)
ax[-1, -1].axis('off')
for i, column in enumerate(CONTINUOUS_FEATURES):
    plot = sns.boxenplot(
        data=df, 
        y=column, 
        x='output', 
        ax=ax[i%rows, i//rows], 
        linewidth=1,
        palette=sns.color_palette('deep'), 
    )
    plot.set_ylabel(column, fontsize=14)
    plot.set(xlabel=None)
fig.suptitle('Boxen plot for each continuous feature split by target', fontsize=16)
fig.tight_layout()
plt.show()

Notes:
* While it may seem intuitive that greater age would correspond with a high likeliness of heart attack, heart attacks skew younger for this data.

In [ ]:
rows = 3
columns = math.ceil(len(CONTINUOUS_FEATURES)/rows)
fig, ax = plt.subplots(
    rows,
    columns,
    figsize=(5*columns, 4*rows),
    sharey=False,
)
ax[-1, -1].axis('off')
for i, column in enumerate(CONTINUOUS_FEATURES):
    sns.kdeplot(
        data=df, 
        x=column, 
        hue='output', 
        ax=ax[i%rows, i//rows],  
        fill=True, 
        linewidth=0,
        palette=sns.color_palette('dark')[:2],
        alpha=.3,
    )
    plot.set_xlabel(column, fontsize=14)
    if i//rows > 0:
        plot.set(ylabel=None)
fig.suptitle('KDE Density plot for each continuous feature split by target', fontsize=16)
fig.tight_layout()

Notes:
* The high frequency of category 0 in oldpeak is predominantly from output 1 samples.

# 2. Data Preparation

## 2.1 Imports

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

## 2.2 Creating training set and test set

In [ ]:
# Copy df before creating dummies for categorical features
df_copy = df.copy(deep=True)

# Converting categorical features into binary categories
df_copy = pd.get_dummies(df_copy, columns=CATEGORICAL_FEATURES, drop_first=True)

# Scaling continuous features using RobustScaler
scaler = RobustScaler()
df_copy[CONTINUOUS_FEATURES] = scaler.fit_transform(df_copy[CONTINUOUS_FEATURES])

# Splitting the data
X = df_copy.drop(['output'], axis=1)
y = df_copy[['output']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=75)
for name, split in {
    'X_train': X_train, 
    'y_train': y_train, 
    'X_test': X_test, 
    'y_test': y_test,
}.items():
    descripter = f'The shape of {name} is:'
    print(f'{descripter:<24} {split.shape}')

# 3. Models

## 3.1 Imports

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from time import time

## 3.2 Defining a function to train, predict, and score a classifier

In [ ]:
def train_and_print_results(classifier, header):
    start_time = time()
    model = classifier.fit(X_train, np.ravel(y_train))
    total_time = time() - start_time

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(header)
    print(f"  accuracy: {accuracy:.3f}")
    print(f'  time: {total_time*100:.3f}ms')

## 3.3 K-Nearest Neighbor Classifier

In [ ]:
# No hyperparameter tuning
classifier = KNeighborsClassifier()
train_and_print_results(classifier, 'KNeighborsClassifier no tuning')

# Parameter tuning
parameters = {
    'n_neighbors': [1, 2, 4, 8, 16, 32, 64],
    'weights': ['uniform', 'distance'],
}
classifier = GridSearchCV(
    KNeighborsClassifier(), 
    parameters,
    cv=3,
)
train_and_print_results(classifier, 'KNeighborsClassifier with tuning')

## 3.4 Support Vector Classifier

In [ ]:
# No tuning
classifier = SVC()
train_and_print_results(classifier, 'SVC no tuning')

# Linear
parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'kernel': ['linear'],
}
classifier = GridSearchCV(
    SVC(),
    parameters,
    cv=3,
)
train_and_print_results(classifier, 'Linear SVC with tuning')

# Poly
parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'kernel': ['poly'],
    'degree': [.5, 2, 3, 4, 5, 10],
    'gamma': ['scale', 'auto'],
}
classifier = GridSearchCV(
    SVC(),
    parameters,
    cv=3,
)
train_and_print_results(classifier, 'Poly SVC with tuning')

# RBF
parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'kernel': ['rbf'],
    'gamma': ['scale', 'auto'],
}
classifier = GridSearchCV(
    SVC(),
    parameters,
    cv=3,
)
train_and_print_results(classifier, 'RBF SVC with tuning')

## 3.5 Stochastic Gradient Descent Classifier

In [ ]:
# No tuning
classifier = SGDClassifier(random_state=75)
train_and_print_results(classifier, 'SGDClassifier no tuning')

# Tuning
parameters = {
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'random_state': [75],
}
classifier = GridSearchCV(
    SGDClassifier(),
    parameters,
    cv=3,
)
train_and_print_results(classifier, 'SGDClassifier with tuning')

# 4. Results
The best performing model was the poly kernal Support Vector Classifier, with an accuracy of 85%. It should be noted that the accuracy is fairly dependent on the random state used to split the training and testing data. This is likely due to the relatively small data set provided, with only 303 samples.